In [2]:
pip install xgboost

     ---------------------------------------- 89.1/89.1 MB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import myutils
from myutils import load_corpus,stopwords


TRAIN_PATH = "C:/Users/11040/Desktop/weibonlp-master/movie/model/train.csv"
TEST_PATH = "C:/Users/11040/Desktop/weibonlp-master/movie/model/test.csv"

train_data = load_corpus(TRAIN_PATH)
test_data = load_corpus(TEST_PATH)

import pandas as pd

df_train = pd.DataFrame(train_data, columns=["words", "label"])
df_test = pd.DataFrame(test_data, columns=["words", "label"])

df_train.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\11040\AppData\Local\Temp\jieba.cache
Loading model cost 0.757 seconds.
Prefix dict has been built successfully.


,words,label
0,原来 人死 后 还要 继续 干 生前 的 活儿 是 真的,1
1,刘德华 那条 线 有点儿 意思 为 这个 系列 电影 开启 了 新 维度 太长 了 删减 强...,1
2,不必 对比 什么 作为 里程碑 他 已经 无敌,1
3,这是 俄罗斯 的 型号 没有 安全 保险 只有 两个 扳机,1
4,年 看 第一部 的 时候 就 想着 回国 之后 如果 能 在 这样 的 剧组 工作 就 好 ...,1


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern='\[?\w+\]?', 
                             stop_words=stopwords,
                             max_features=2000)

X_train = vectorizer.fit_transform(df_train["words"])
y_train = df_train["label"]
X_test = vectorizer.transform(df_test["words"])
y_test = df_test["label"]

C:\Users\11040\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['元', '吨', '数', '末'] not in stop_words.
  warnings.warn(


In [69]:
import xgboost as xgb

param = {
    'booster':'gbtree',
    'learning_rate':0.6,
    'max_depth': 9, 
    'min_child_weight':1,
    'gamma':0.2,
    'subsample':0.9,
    'scale_pos_weight': 0.6,
    'colsample_bytree': 0.8,
    'objective': 'binary:logistic',
    'eval_metric': 'error',
    'eta': 0.3,
    'nthread': 10,
}
dmatrix = xgb.DMatrix(X_train, label=y_train)
model = xgb.train(param, dmatrix, num_boost_round=200)

In [70]:
dmatrix = xgb.DMatrix(X_test)
y_pred = model.predict(dmatrix)

from sklearn import metrics

auc_score = metrics.roc_auc_score(y_test, y_pred)          
y_pred = list(map(lambda x:'1' if x > 0.5 else '0', y_pred))  

# 测试集效果检验
print(metrics.classification_report(y_test, y_pred))
print("准确率:", metrics.accuracy_score(y_test, y_pred))
print("AUC:", auc_score)

              precision    recall  f1-score   support

           0       0.80      0.51      0.62        69
           1       0.78      0.93      0.85       131

    accuracy                           0.79       200
   macro avg       0.79      0.72      0.73       200
weighted avg       0.79      0.79      0.77       200

准确率: 0.785
AUC: 0.8027436663347716


In [78]:
from myutils import processing

strs = ["看吐了 难看爆了", "特效真绝！太空电梯好酷炫！"]
words = [processing(s) for s in strs]
vec = vectorizer.transform(words)
dmatrix = xgb.DMatrix(vec)
output = model.predict(dmatrix)
print(output)

[0.03304209 0.857257  ]
